In [13]:
# 75.06 - 1C202 - TP2 
# Competencia de Machine learning
#
# https://www.kaggle.com/c/nlp-getting-started/overview
#
# Fuentes:
# https://realpython.com/python-keras-text-classification/
# https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28

# Importacion de librerias necesarias
import re, string, random, datetime
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Split y K-Fold 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Importacion de Keras
# https://keras.io
from keras.models import Model, Sequential, save_model, load_model
from keras.layers import Embedding, Conv1D, Dropout, Input, GlobalMaxPooling1D, Dense, concatenate, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from bert import bert_tokenization


#Configuracion general
plt.style.use('ggplot')
warnings.filterwarnings('ignore')

import tokenization

In [14]:
# Path de entrada
train_path = 'data/processed/train.2020.08.03T11.00.50.547958.csv' #Cambiar por el archivo pre-procesado a usar!
test_path = 'data/processed/test.2020.08.03T11.00.50.547958.csv' #Cambiar por el archivo pre-procesado a usar!

# Configuracion
maxlen = 100

# Carga de los archivos del set de datos
# Set de train: carga
df_twitter_train = pd.read_csv(train_path, sep=',')
# Set de test: carga
df_twitter_test = pd.read_csv(test_path, sep=',')

# Print de los shapes
print('Shape train: ' + str(df_twitter_train.shape))
print('Shape test: ' + str(df_twitter_test.shape))

Shape train: (7613, 4)
Shape test: (3263, 3)


In [15]:
df_twitter_train.head()

,id,keyword,text,target
0,1,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,forest fire near la ronge sask canada,1
2,5,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,people receive wildfires evacuation orders in...,1
4,7,NaN,just got sent this photo from ruby alaska as s...,1


In [16]:
df_twitter_test.head()

,id,keyword,text
0,0,NaN,just happened a terrible car crash
1,2,NaN,heard about earthquake is different cities sta...
2,3,NaN,there is a forest fire at spot pond geese are ...
3,9,NaN,apocalypse lighting spokane wildfires
4,11,NaN,typhoon soudelor kills in china and taiwan


In [17]:
#defino funcion bert_encode

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [18]:
# Construimos el modelo
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
#Cargamos BERT desde tensorflow

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [20]:
# Tokenizamos

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [21]:
# Nos quedamos con X e y para entrenar
tweets = df_twitter_train['text'].values
y = df_twitter_train['target'].values

# Separamos X para el set de test
tweets_predict = df_twitter_test['text'].values

train_encode_bert = bert_encode(tweets, tokenizer, max_len=160)
test_encode_bert = bert_encode(tweets_predict, tokenizer, max_len=160)
values_to_predict = df_twitter_train['target'].values

In [22]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [23]:
# Entrenamos el modelo
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_encode_bert, values_to_predict,
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint],
    batch_size=2
)
#probar batch_size = 15

3045/3045 [==============================] - 21181s 7s/step - loss: 0.4447 - accuracy: 0.8087 - val_loss: 0.3816 - val_accuracy: 0.8365


In [24]:
# Mostramos el historial del entrenamiento
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.444715,0.808703,0.381563,0.836507


In [25]:
# Hacemos la prediccion

model.load_weights('model.h5')
y_predict = model.predict(test_encode_bert)

In [26]:
# Generamos el submit
kaggle_submission = pd.DataFrame(df_twitter_test,columns = ['id'])
kaggle_submission['target'] = (y_predict > 0.5).astype('int32')

submission_path = 'data/submits/submission.' + datetime.datetime.now().isoformat() + '.csv'
submission_path = submission_path.replace('-','.').replace(':','.')

kaggle_submission.to_csv(submission_path, index=False)

# Imprimimos un resumen de la operacion
print('Operación finalizada!\n')
print('Generado submit: \'' + submission_path + '\' - (' + str(len(kaggle_submission['target'].index)) + ') registros.')

Operación finalizada!

Generado submit: 'data/submits/submission.2020.08.06T16.39.18.657716.csv' - (3263) registros.
